# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770839461338                   -0.52    9.0         
  2   -2.772146254115       -2.88       -1.33    1.0    201ms
  3   -2.772170093618       -4.62       -2.40    1.0    234ms
  4   -2.772170637219       -6.26       -3.10    1.0    207ms
  5   -2.772170722632       -7.07       -4.39    2.0    294ms
  6   -2.772170722871       -9.62       -4.61    1.0    204ms
  7   -2.772170723006       -9.87       -5.16    2.0    242ms
  8   -2.772170723013      -11.14       -5.56    1.0    228ms
  9   -2.772170723015      -11.77       -6.35    2.0    247ms
 10   -2.772170723015      -13.75       -6.77    1.0    282ms
 11   -2.772170723015      -13.94       -7.72    1.0    235ms
 12   -2.772170723015      -14.88       -8.45    2.0    287ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735581436821426

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770756389490                   -0.52    9.0         
  2   -2.772060138740       -2.88       -1.32    1.0    189ms
  3   -2.772082987809       -4.64       -2.43    1.0    272ms
  4   -2.772083336006       -6.46       -3.12    1.0    197ms
  5   -2.772083417615       -7.09       -4.32    2.0    232ms
  6   -2.772083417743       -9.89       -4.67    1.0    267ms
  7   -2.772083417809      -10.18       -5.74    1.0    211ms
  8   -2.772083417811      -11.72       -6.73    2.0    285ms
  9   -2.772083417811      -14.15       -6.67    1.0    229ms
 10   -2.772083417811   +  -14.88       -7.91    1.0    248ms
 11   -2.772083417811   +  -14.57       -8.34    2.0    283ms

Polarizability via ForwardDiff:       1.7725349777181945
Polarizability via finite difference: 1.7735581436821426
